In [34]:
import pandas as pd

In [68]:
mixed_type_columns = [3, 5, 11, 28, 362, 392, 397, 402, 407, 412, 417, 437, 442, 447, 452, 457, 472, 477, 482, 492, 497, 502, 507, 517, 522, 527, 532, 542, 547, 552, 557, 562, 567, 572, 577, 582, 587, 592, 597, 602, 607, 612, 617, 622, 627, 632, 637, 642, 647, 652, 655, 668]

dtype_dict = {col: str for col in mixed_type_columns}

openei = pd.read_csv('/Users/caseychens/Documents/stanford/summer24/we3/data/OpenEI data/91761.csv', dtype=dtype_dict)

openei['sourceparent'] = openei['sourceparent'].fillna('')

data_dict = {
    'utility': [],
    'type': [],
    'assessed': [],
    'period': [],
    'basic_charge_limit (imperial)': [],
    'basic_charge_limit (metric)': [],
    'month_start': [],
    'month_end': [],
    'hour_start': [],
    'hour_end': [],
    'weekday_start': [],
    'weekday_end': [],
    'charge (imperial)': [],
    'charge (metric)': [],
    'units': [],
    'Notes': []
}


In [69]:
index = 0

#for i in range(len(openei_dummy)):
for i in range(1):
    # customer flat rate (need to add code checking if rate is zero)
    data_dict['utility'].append('electricity')
    data_dict['type'].append('customer')
    data_dict['assessed'].append('')
    data_dict['period'].append('')
    data_dict['basic_charge_limit (imperial)'].append('')
    data_dict['basic_charge_limit (metric)'].append('')
    data_dict['month_start'].append('')
    data_dict['month_end'].append('')
    data_dict['hour_start'].append('')
    data_dict['hour_end'].append('')
    data_dict['weekday_start'].append('')
    data_dict['weekday_end'].append('')
    data_dict['charge (imperial)'].append(openei['fixedchargefirstmeter'][i])
    data_dict['charge (metric)'].append(openei['fixedchargefirstmeter'][i])
    data_dict['units'].append('$/month')
    data_dict['Notes'].append(openei['source'][i] + ('\t' + openei['sourceparent'][i] if openei['sourceparent'][i] != '' else ''))
    
    # First demand tariff
    start_elec = 31
    end_elec = 99
    num_columns = 4
    for j in range(start_elec, end_elec, num_columns):
        data_dict['utility'].append('electricity')
        data_dict['type'].append('demand')
        data_dict['assessed'].append('')
        data_dict['period'].append('')
        data_dict['basic_charge_limit (imperial)'].append('')
        data_dict['basic_charge_limit (metric)'].append('')
        data_dict['month_start'].append('')
        data_dict['month_end'].append('')
        data_dict['hour_start'].append('')
        data_dict['hour_end'].append('')
        data_dict['weekday_start'].append('')
        data_dict['weekday_end'].append('')
        data_dict['charge (imperial)'].append(openei['flatdemandstructure/period0/tier0rate'][i] + openei['flatdemandstructure/period0/tier1adj'][i])
        data_dict['charge (metric)'].append(openei['flatdemandstructure/period0/tier0rate'][i]+ openei['flatdemandstructure/period0/tier1adj'][i])
        data_dict['units'].append('$/kW')
        data_dict['Notes'].append('')
    
    #Successive tariffs batched in sets of 4
    #for j in range(32)



    data_dict['utility'].append('electricity')
    data_dict['type'].append('demand')
    data_dict['assessed'].append('')
    data_dict['period'].append('')
    data_dict['basic_charge_limit (imperial)'].append('')
    data_dict['basic_charge_limit (metric)'].append('')
    data_dict['month_start'].append('')
    data_dict['month_end'].append('')
    data_dict['hour_start'].append('')
    data_dict['hour_end'].append('')
    data_dict['weekday_start'].append('')
    data_dict['weekday_end'].append('')
    data_dict['charge (imperial)'].append(openei['flatdemandstructure/period0/tier0rate'][i])
    data_dict['charge (metric)'].append(openei['flatdemandstructure/period0/tier0rate'][i])
    data_dict['units'].append('$/kW')
    data_dict['Notes'].append('')

for key, value in data_dict.items():
    print(f"{key}: {len(value)}")

new_df = pd.DataFrame(data_dict)

utility: 19
type: 19
assessed: 19
period: 19
basic_charge_limit (imperial): 19
basic_charge_limit (metric): 19
month_start: 19
month_end: 19
hour_start: 19
hour_end: 19
weekday_start: 19
weekday_end: 19
charge (imperial): 19
charge (metric): 19
units: 19
Notes: 19
